# Práctica 1

## Preparación del entorno

In [1]:
import numpy as np
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os
import random

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm

### Variables a definir

In [2]:
# No tocar
num_clases = 10
# La foto es 128x96
xpixel = 128
ypixel = 96
# Tocar
n_neuronas_conv1 = 64
n_neuronas_conv2 = 128
n_neuronas_conv3 = 256
l_rate = 0.15
epoch = 15 
batch = 1

# Preprocesamiento de datos

### Esto coge los datos originales y los divide parece funcionar para model.fit

In [22]:
# Esto funciona con fit pero necesito investigar

# Mi  teoria de lo que es esto por lo que he leido (OJO NO LO SE REALMENTE) esto crea iteradores que formatean la imagen y la pasan por al model.fit el batch size es en cuanto pedazos pasas
# por cadda iteracion como diferencia y hace el split entre validadcion y training no lose pero salia asi en bastantes sitios 

batch_size = 1 

# this is the augmentation configuration we will use for training     ESTO define como pasas las imagenes puedes modificarlas con parametros yo solo normalizo
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        validation_split=0.2,
        )

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator_t = train_datagen.flow_from_directory(                        # Esto creo que crea un iterador en el directorio que sige las intrucciones mencioanda arriba para pasar los                                                                        # datos a model.fit =/ nose es una paranoia
        directory=r'../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(xpixel, ypixel),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="training")  # since we use binary_crossentropy loss, we need binary labels

train_generator_v = train_datagen.flow_from_directory(
        directory=r'../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(xpixel, ypixel),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="validation") 

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


### Mismo Concepto pero para test ahora en construccion

In [23]:
batch_size = 1 
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=r'../Dataset/imgs/test/',

    color_mode='rgb',
    batch_size=batch_size,
    image_size=(xpixel, ypixel),
    shuffle=True,
    seed=123,
)

Found 0 files belonging to 0 classes.


ValueError: No images found in directory ../Dataset/imgs/test/. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

##### Otra Red para no borrar lo ya hecho

In [10]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model = Sequential()

    #### Input Layer ####
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',
                    activation='relu', input_shape=(128, 96, 3)))

    #### Convolutional Layers ####
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))  # Pooling
    model.add(Dropout(0.2)) # Dropout

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(ker.layers.Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
    model.add(Conv2D(512, (5,5), activation='relu'))
    model.add(MaxPooling2D((4,4)))
    model.add(Dropout(0.2))

    #### Fully-Connected Layer ####
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax')) #len(class_subset) en vez de 10, lo he cambiado pero no estoy seguro

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    model.summary() # a handy way to inspect the architecture
    return model

### Red OG          EJECUTAR ESTA

In [5]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model=Sequential()
    # Capa input
    model.add(Conv2D(filters= n_conv1, kernel_size=(1,1),input_shape=(x,y,3),padding='same',activation='relu')) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    # (3,3) es mucho se recomienda unsar 1x1 cuando las img no son mayores de 128x128 la nuestra es 128x96
    model.add(Conv2D(n_conv2,(1,1),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(1,1),activation='relu',padding='same')) 
    model.add(Conv2D(n_conv3,(1,1),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected - MLP
    model.add(Flatten())
    model.add(Dense(512,activation='relu')) # red fully-connected
    model.add(Dropout(0.5))
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo

    # Compilamos
    adam = tf.keras.optimizers.Adam(learning_rate= lr) # https://keras.io/api/optimizers/adam/ ?
    model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])

    return model
model = Modelar_red(n_neuronas_conv1,n_neuronas_conv2,n_neuronas_conv3,xpixel,ypixel,l_rate)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 96, 64)       256       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 96, 128)      8320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 48, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 48, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 48, 256)       33024     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 48, 256)       65792     
                                                        

### Y ahora realizamos el fit para entrenar la red

In [7]:
def Entrenar(m,e,v,epo,b):
 #(model,entrenamiento,validacion, epoch,batch)
        m.fit(
        e,
        validation_data=v,
        epochs= epo,
        batch_size = b, 
        verbose=1 #Esto te imprime un progress bar con informacion 
        ) 
        return m
model = Entrenar(model,train_generator_t,train_generator_v,epoch,batch)
# DATO CURIOSO, .FIT() TIENE UNA OPCION LLAMADA VALIDATION_DATA=, IGUAL NI ES NECESARIO CREAR UN GRUPO DE VALIDACION A PARTE

Epoch 1/15
17943/17943 [==============================] - 531s 29ms/step - loss: 16707.3906 - accuracy: 0.1030 - val_loss: 2.5330 - val_accuracy: 0.1033
Epoch 2/15
17943/17943 [==============================] - 548s 31ms/step - loss: 2.5273 - accuracy: 0.0979 - val_loss: 2.4805 - val_accuracy: 0.1047
Epoch 3/15
17943/17943 [==============================] - 555s 31ms/step - loss: 2.5350 - accuracy: 0.1014 - val_loss: 2.6751 - val_accuracy: 0.1011
Epoch 4/15
17943/17943 [==============================] - 550s 31ms/step - loss: 2.5235 - accuracy: 0.1013 - val_loss: 2.4937 - val_accuracy: 0.1038
Epoch 5/15
 2396/17943 [===>..........................] - ETA: 7:48 - loss: 2.5256 - accuracy: 0.1010

KeyboardInterrupt: 

#### Cargar Modelo

In [2]:
model = ker.models.load_model('path/to/location') # Carga modelos

NameError: name 'ker' is not defined

#### Guardar Modelo

In [8]:
model.save('modelo2') # guarda el modelo en la ruta que desees

INFO:tensorflow:Assets written to: modelo2\assets


In [9]:
model.evaluate(train_generator_v)

4481/4481 [==============================] - 31s 7ms/step - loss: 2.5014 - accuracy: 0.1109


[2.501357316970825, 0.1109127402305603]

#### Evaluar Modelo

In [ ]:
def Evaluar(m,e_test,s_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,s_test,verbose=False)
resultado = Evaluar(model, entrada_test, salida_test) # no existen estas variables todavia

#### Predecir

In [ ]:
prediccion = model.predict(train_generator_test) # no existe esta variable todavia
print(prediccion)

ValueError: Asked to retrieve element 0, but the Sequence has length 0